In [68]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
import tools
from math import sqrt
from script.tools import tools
#from script.collection import Parameter as pmt
import sys
print(sys.executable)
import net
import os
from script.tools import collection
#result = collection.collection("collection/workspace1")  #[iops,dic_ssd,dic_worload,f.path]
#result = collection.collection("collection/workspace2",result)
result = collection.collection("collection/workspace_ssd0_workload0")
#result = result[:1000]
plst = collection.getParameters(path="collection/config.xlsx",id = "0")
expect = ['Seed','key','default']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/opt/homebrew/anaconda3/bin/python
no scenario collection/workspace_ssd0_workload0/1854/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/425/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/866/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/1994/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/1702/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/428/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/3472/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/1796/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/2148/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/2918/workload_scenario_1.xml
no scenario collection/workspace_ssd0_workload0/3981/workload_scenario_1.xml
no scenario collection/workspace_

In [69]:
def test(x_,y_,m, mean = 0,var = 1):
    x_ = (x_ - mean)/var
    y_pre = m(x_)
    y_e = abs(y_pre - y_)/y_
    print(torch.sum(y_e)/len(y_))

In [70]:
#print(result[0])
from script.tools import collection
split_index = int(len(result)*0.7)
train_result = result[:split_index]
test_result  = result[split_index:]
x_train_lst, y_train_lst, keys = collection.result2lst(train_result,plst,expect)
x_test_lst,  y_test_lst, keys = collection.result2lst(test_result,plst,expect)
x_train_tensor = torch.tensor(  x_train_lst  , dtype=torch.float)
y_train_tensor = torch.tensor(  y_train_lst  , dtype=torch.float)/10000
x_test_tensor =  torch.tensor(  x_test_lst   , dtype=torch.float)
y_test_tensor =  torch.tensor(  y_test_lst   , dtype=torch.float)/10000
x_mean = x_train_tensor.mean(dim = 0)
x_var = x_train_tensor.var(dim = 0)
x_train_tensor_nor = (x_train_tensor -x_mean)/x_var
# x用正则化后的数据，y除以10000，得到模型f(x-u)
dataset = collection.CustomDataset(x_train_tensor_nor, y_train_tensor)
batch_size = 5
dataloader = collection.DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = net.DeepNet(19,8,1,5)
criterion = nn.MSELoss()
print(len(keys))
print(keys)
print(x_train_tensor.shape)

19
['Overprovisioning_Ratio', 'Flash_Channel_Count', 'Channel_Transfer_Rate', 'Chip_No_Per_Channel', 'Page_Read_Latency_LSB', 'Page_Read_Latency_MSB', 'Page_Program_Latency_LSB', 'Page_Program_Latency_MSB', 'Die_No_Per_Chip', 'Plane_No_Per_Die', 'Block_No_Per_Plane', 'Page_No_Per_Block', 'Channel_IDs', 'Chip_IDs', 'Die_IDs', 'Plane_IDs', 'Initial_Occupancy_Percentage', 'Working_Set_Percentage', 'Read_Percentage']
torch.Size([2671, 19])


In [37]:
import torch
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
def lasso(x,y):
    x_np = x.detach().numpy()
    y_np = y.detach().numpy().ravel()
    param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
    lasso = Lasso()
    grid_search = GridSearchCV(lasso, param_grid, cv=5)
    grid_search.fit(x_np, y_np)
    best_alpha = grid_search.best_params_['alpha']
    lasso_best = Lasso(alpha=best_alpha)
    lasso_best.fit(x_np, y_np)
    nonzero_coeff_indices = [i for i, coeff in enumerate(lasso_best.coef_) if coeff != 0]
    x_reduced_np = x_np[:, nonzero_coeff_indices]
    x_reduced = torch.tensor(x_reduced_np, dtype=torch.float32)


In [44]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 200
lambda_l2 = 2e-6
loss_history = []
for epoch in range(num_epochs):
    total_loss = 0.0
    i = 0
    for batch_x, batch_y in dataloader: # x_train_tensor_nor
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        l2_regularization = torch.tensor(0.)
        for param in model.parameters():
            l2_regularization += torch.norm(param, p=2)
            loss = loss + l2_regularization * lambda_l2
        i = i + 1
        if i == -1:
            print("i=------"+str(i),"\n outputs: ",outputs,"\n y:",batch_y)
        total_loss += loss.item()
        if False:
            print("   \n \n")
            print(" outputs: ", outputs)
            print(" batch_y", batch_y)
            print(" loss ", loss)
            print("   \n \n")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        average_loss = total_loss / len(dataloader)
        #print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.6f}')
        print("test:")
        test(x_test_tensor,y_test_tensor,model,x_mean,x_std) # test准确度

Epoch [100/200], Loss: 0.068887
test:
tensor(6.0088, grad_fn=<DivBackward0>)
R2
Epoch [200/200], Loss: 0.048858
test:
tensor(15.3267, grad_fn=<DivBackward0>)
R2


In [62]:
print(x_train_tensor_nor[0])
# 训练集
print("train")
test(x_train_tensor,y_train_tensor,model,x_mean,x_std)
print("\n \n test:")
test(x_test_tensor,y_test_tensor,model,x_mean,x_std) # test
#print(x_train_tensor[0],model(x_train_tensor_nor[0]),y_train_tensor[0])
print("\n \n")
if 1 :
    id = 762
    print("out:",y_test_tensor[id])
    print(model( (x_test_tensor-x_mean)/x_var )[id])
print(len(keys))
print(keys)
print(type(x_train_lst[0]))
print(y_train_lst[0])

tensor([ 2.9702e+01,  6.7141e-01,  7.5537e-04, -4.7359e-01,  5.0434e-06,
        -4.4459e-06,  5.2011e-08,  6.5974e-07, -6.7162e-01,  3.0209e-01,
         7.6860e-04,  3.5376e-03,  9.5929e-02, -2.2362e-01, -5.2445e-01,
         1.0061e+00,  2.6238e-02, -3.2114e-02, -9.4298e-03])
train
tensor(15.4857, grad_fn=<DivBackward0>)

 
 test:
tensor(15.3267, grad_fn=<DivBackward0>)

 

out: tensor([0.6269])
tensor([0.8681], grad_fn=<SelectBackward0>)
19
<class 'list'>
[2331.9]


In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


# 假设 x_train_lst 和 y_train_lst 已经被定义并且格式正确
x_train = x_train_lst
y_train = [y[0] for y in y_train_lst]  # 将 y_train_lst 转换为一维列表

# 创建线性回归模型
model = LinearRegression()

# 训练模型
model.fit(x_train, y_train)

# 进行预测（以 x_train 为例）
y_pred = model.predict(x_train)

# 打印预测结果
print(y_pred)
weights = model.coef_

# 打印权重
print("权重（斜率系数）:\n", weights)

# 如果你也想获取截距
intercept = model.intercept_
print("截距（Intercept）:\n", intercept)
mse = mean_squared_error(y_train, y_pred)
print("mse:",mse)
r2 = r2_score(y_train, y_pred)
print("决定系数 (R²):", r2)

[ 2368.34544935 -2045.66728335  3242.3426917  ...  5606.53286947
  5468.93304925  4450.27732164]
权重（斜率系数）:
 [-4.17607650e+02  4.32997828e+01  1.04073694e-01  4.46306813e+01
 -2.40524106e-03 -2.40163519e-03 -3.31336743e-04 -3.22609545e-04
  4.40879654e+01  1.03167687e+01 -5.13006518e-02 -1.80649423e-02
  9.62653185e+02  9.55254700e+02  2.41831674e+01  1.82711512e+02
  4.73006850e-02 -7.94264185e-01  2.78277420e+01]
截距（Intercept）:
 393.19889208686345
mse: 7009074.059436485
决定系数 (R²): 0.5489657530453222


In [44]:
xlst,ylst,keys = collection.result2lst(result,plst,expect)
test_result = collection.collection("collection/workspace")
x_test_lst,y_test_lst,keys = collection.result2lst(test_result,plst,expect)
x_tensor = torch.tensor(xlst,dtype=torch.float)
y_tensor = torch.tensor(ylst,dtype=torch.float)
x_test_tensor =  torch.tensor(x_test_lst,dtype = torch.float)
y_test_tensor =  torch.tensor(y_test_lst,dtype = torch.float)

model = net.DeepNet(48,48,1,5)
criterion = nn.MSELoss()


no scenario collection/workspace/28/workload_scenario_1.xml
no scenario collection/workspace/30/workload_scenario_1.xml
collection:  6
total : 8


In [107]:
model = net.DeepNet(48,48,1,5)
#device = torch.device("mps")
#device = torch.device("cpu")
#model.to(device)
criterion = nn.MSELoss()

model_0.0982.pth  exists


<All keys matched successfully>

In [108]:
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# dataset = CustomDataset(x_tensor, y_tensor)
y_modified = y_tensor/y_mean
dataset = CustomDataset(normalized_x, y_modified)
batch_size = 5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#optimizer = optim.SGD(model.parameters(), lr=0.001)




tensor([[0.5618],
        [0.1125],
        [0.2550],
        [0.0654],
        [0.0771],
        [0.3035],
        [0.4796],
        [0.3741],
        [0.1920],
        [0.1301],
        [0.1324],
        [0.4283],
        [0.1418],
        [0.2323],
        [0.2001],
        [0.0292],
        [0.2910],
        [0.5038],
        [0.0850],
        [0.6712],
        [0.5066],
        [1.9664],
        [0.0620],
        [0.1514],
        [0.5173],
        [0.3189],
        [0.6853],
        [0.4222],
        [0.2690],
        [0.2659],
        [0.0697],
        [0.4649],
        [0.3005],
        [0.7437],
        [0.2272],
        [0.3342],
        [0.2523],
        [0.8178],
        [0.1258],
        [0.1817],
        [0.6557],
        [0.3641],
        [0.0528],
        [0.6096],
        [0.2518],
        [0.9682],
        [0.1117],
        [0.3666],
        [0.0466],
        [0.1309],
        [0.2234],
        [0.1421],
        [0.2673],
        [0.4538],
        [0.1944],
        [1

In [52]:
optimizer = optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 1000

loss_history = []
for epoch in range(num_epochs):
    total_loss = 0.0
    i = 0
    for batch_x, batch_y in dataloader:
        #print("hhhh ")
        #print(batch_x,batch_x)
        #i = i +1
        #print(batch_x,"\n",batch_y)
        #print("zzzzzzz")
        # 前向传播

        outputs = model(batch_x)
        #print("batch_x",batch_x[0],"\n outputs: ",outputs[0])
        loss = criterion(outputs, batch_y)
        i = i +1
        if i == -1:
            print("i=------"+str(i),"\n outputs: ",outputs,"\n y:",batch_y)
        total_loss += loss.item()
        if False:
            print("   \n \n")
            print(" outputs: ", outputs)
            print(" batch_y", batch_y)
            print(" loss ", loss)

            print("   \n \n")
        #print(f'Epoch [{epoch+1}/{num_epochs}]')
        #print("loss:",loss.item(),"batch_x: ",batch_x," outputs ",outputs," batch_y: ",batch_y)
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #loss_history.append(total_loss)
    # 打印训练信息
    if (epoch+1) % 10 == 0:
        average_loss = total_loss / len(dataloader)
        #print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.6f}')

def test(x_,y_,m,rate_y, mean_x = 0,std_x = 1):
    x_ = (x_ - mean_x)/std_x
    y_pre = m(x_)*rate_y
    y_e = abs(y_pre - y_)/y_
    print(torch.sum(y_e)/len(y_))
# 训练集
test(x_tensor,y_tensor,model,y_mean,x_mean,x_std)

print("\n \n test:")
test(x_test_tensor,y_test_tensor,model,y_mean,x_std)



Epoch [10/1000], Loss: 0.000246
Epoch [20/1000], Loss: 0.000245
Epoch [30/1000], Loss: 0.000244
Epoch [40/1000], Loss: 0.000242
Epoch [50/1000], Loss: 0.000242
Epoch [60/1000], Loss: 0.000240
Epoch [70/1000], Loss: 0.000240
Epoch [80/1000], Loss: 0.000239
Epoch [90/1000], Loss: 0.000237
Epoch [100/1000], Loss: 0.000237
Epoch [110/1000], Loss: 0.000236
Epoch [120/1000], Loss: 0.000235
Epoch [130/1000], Loss: 0.000235
Epoch [140/1000], Loss: 0.000234
Epoch [150/1000], Loss: 0.000233
Epoch [160/1000], Loss: 0.000232
Epoch [170/1000], Loss: 0.000231
Epoch [180/1000], Loss: 0.000229
Epoch [190/1000], Loss: 0.000229
Epoch [200/1000], Loss: 0.000228
Epoch [210/1000], Loss: 0.000228
Epoch [220/1000], Loss: 0.000226
Epoch [230/1000], Loss: 0.000225
Epoch [240/1000], Loss: 0.000225
Epoch [250/1000], Loss: 0.000223
Epoch [260/1000], Loss: 0.000223
Epoch [270/1000], Loss: 0.000221
Epoch [280/1000], Loss: 0.000220
Epoch [290/1000], Loss: 0.000220
Epoch [300/1000], Loss: 0.000219
Epoch [310/1000], L

In [57]:
import numpy as np
y_predict_train = model(normalized_x)
y_err = (y_predict_train - y_modified)/y_modified
print(torch.sum(abs(y_err))/len(y_predict))

def test(x_,y_,m,rate_y, mean_x = 0,std_x = 1):
    x_ = (x_ - mean_x)/std_x
    y_pre = m(x_)*rate_y
    y_e = abs(y_pre - y_)/y_
    print(torch.sum(y_e)/len(y_))
# 训练集
test(x_tensor,y_tensor,model,y_mean,x_mean,x_std)

print("\n \n test:")
test(x_test_tensor,y_test_tensor,model,y_mean,x_std)
#print(len(x_tensor))
# 测试集



tensor(0.2114, grad_fn=<DivBackward0>)
tensor(0.2114, grad_fn=<DivBackward0>)

 
 test:
tensor(1.3245, grad_fn=<DivBackward0>)


In [16]:
result1 = []
result1 = collection.collection("collection/workspace",result1)

collection:  1070


In [48]:
#print(xlst[0])
print(len(xlst[0]))
tx = []
tx.append(keys)
i = 0
for y_ in ylst:
    x_ = xlst[i].copy()
    x_.append(y_[0])
    tx.append(x_)
    i = i + 1
print(tx[0])
tools.lst2excel(tx,"test.xlsx")

48
['PCIe_Lane_Bandwidth', 'PCIe_Lane_Count', 'SATA_Processing_Delay', 'Queue_Fetch_Size', 'Data_Cache_DRAM_Row_Size', 'Data_Cache_DRAM_Data_Rate', 'Data_Cache_DRAM_Data_Busrt_Size', 'Data_Cache_DRAM_tRCD', 'Data_Cache_DRAM_tCL', 'Data_Cache_DRAM_tRP', 'Overprovisioning_Ratio', 'GC_Exec_Threshold', 'Preferred_suspend_erase_time_for_read', 'Preferred_suspend_erase_time_for_write', 'Preferred_suspend_write_time_for_read', 'Flash_Channel_Count', 'Flash_Channel_Width', 'Channel_Transfer_Rate', 'Chip_No_Per_Channel', 'Page_Read_Latency_LSB', 'Page_Read_Latency_CSB', 'Page_Read_Latency_MSB', 'Page_Program_Latency_LSB', 'Page_Program_Latency_CSB', 'Page_Program_Latency_MSB', 'Block_Erase_Latency', 'Block_PE_Cycles_Limit', 'Suspend_Erase_Time', 'Suspend_Program_Time', 'Die_No_Per_Chip', 'Plane_No_Per_Die', 'Block_No_Per_Plane', 'Page_No_Per_Block', 'Page_Capacity', 'Page_Metadat_Capacity', 'Channel_IDs', 'Chip_IDs', 'Die_IDs', 'Plane_IDs', 'Initial_Occupancy_Percentage', 'Working_Set_Percentag

In [135]:
print(len(keys))
import platform

print("Python版本:", platform.python_version())

48
Python版本: 3.9.13
